# SAMME
Adaboost算法的扩展

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from utils.dataset_utils import get_distribution
from instance_selection.parameter.parameter import *  # 导入参数的设定
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import scipy.io as sio  # 从.mat文件中读取数据集
import warnings

warnings.filterwarnings("ignore")  # 忽略警告

DATASET = Satellite  # 数据集名称（包含对应参数的字典形式）
datasetname = DATASET['DATASETNAME'].split('.')[0]

# 加载、划分数据集
mat_data = sio.loadmat(IMBALANCED_DATASET_PATH + DATASET['DATASETNAME'])
x = mat_data['X']
y = mat_data['Y'][:, 0]  # mat_data['Y']得到的形状为[n,1]，通过[:,0]，得到形状[n,]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=RANDOM_SEED)  # 划分数据集
scaler = StandardScaler()  # 数据的标准化
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

unique_elements_all, classes_all, counts_all = get_distribution(y)  # 获取原始数据集分布
unique_elements_train, classes_train, counts_train = get_distribution(y_train)  # 获取训练集分布
unique_elements_test, classes_test, counts_test = get_distribution(y_test)  # 获取测试集分布
print(datasetname + f' distribution: {counts_all}')
print(f'trainset distribution: {counts_train}')
print(f'testset distribution: {counts_test}')

#base_classifier = CustomMLPClassifier(hidden_layer_sizes=(DATASET['HIDDEN_SIZE'],), max_iter=DATASET['MAX_ITER'],
#                                random_state=RANDOM_SEED, learning_rate_init=DATASET['LEARNING_RATE'])

base_classifier = DecisionTreeClassifier(max_depth=1)

In [ ]:
from instance_selection.operator.metrics import calculate_gmean_mauc

# 初始化 AdaBoostClassifier，选择 SAMME 或 SAMME.R
adaboost_samme = AdaBoostClassifier(estimator=base_classifier, n_estimators=40, algorithm="SAMME")
adaboost_samme_r = AdaBoostClassifier(estimator=base_classifier, n_estimators=40, algorithm="SAMME.R")

# 训练模型
adaboost_samme.fit(x_train, y_train)
adaboost_samme_r.fit(x_train, y_train)

# 预测
y_pred_samme = adaboost_samme.predict_proba(x_test)


# 输出预测的准确度量
gmean, mauc ,recall_per_class= calculate_gmean_mauc(y_pred_samme, y_test)
print(f'Gmean: {gmean}: mAUC: {mauc}\n')